In [1]:
import pandas
import time
import numpy as np
import preprocessors

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import accuracy_score
from sklearn.decomposition import IncrementalPCA

from estimators import Bayes_Wrapper

In [2]:
path = "/media/alexander/b32bf4b4-8724-4107-9d19-abf6615c2f60/alexander/HELP_FILE/query.yaHotelId.showInTop.sure.final.tsv"

In [3]:
df = pandas.read_csv(path, sep="\t")
print("Изначальная размерность данных:", df.shape,";", "Количество отелей:", len(df["yaHotelId"].unique()))

Изначальная размерность данных: (588232, 4) ; Количество отелей: 95884


In [4]:
sure_df = df[df["sure"]]
filtered_values = [value[0] for value in sure_df["yaHotelId"].value_counts().iteritems() if value[1] >= 5]
filtered_df = sure_df[sure_df["yaHotelId"].isin(filtered_values)]
print("Получившаяся размерность данных:", filtered_df.shape, ";", "Количество отелей:", len(filtered_df["yaHotelId"].unique()))

Получившаяся размерность данных: (92427, 4) ; Количество отелей: 7696


Разделяем выборку на тренировочную и тестовую части:

In [5]:
df_train, df_test = train_test_split(filtered_df, test_size=0.1)
df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

Преобразуем тренировочное множество:

Tf-idf преобразование:

In [6]:
prep = preprocessors.Preprocessor([preprocessors.mystem_preprocess, preprocessors.dictionary_preprocess])
vectorizer = TfidfVectorizer(preprocessor=prep.preprocess)
y_train = np.array(df_train["yaHotelId"])
X_train = vectorizer.fit_transform(df_train["query"])
X_train.shape

(83184, 12071)

In [7]:
y_test = np.array(df_test["yaHotelId"])
X_test = vectorizer.transform(df_test["query"])

In [8]:
"""
X_train = np.load("X_train.npy")
X_test = np.load("X_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")

np.save("X_train.npy", X_train)
np.save("X_test.npy", X_test)
np.save("y_train.npy", y_train)
np.save("y_test.npy", y_test)
"""

'\nX_train = np.load("X_train.npy")\nX_test = np.load("X_test.npy")\ny_train = np.load("y_train.npy")\ny_test = np.load("y_test.npy")\n\nnp.save("X_train.npy", X_train)\nnp.save("X_test.npy", X_test)\nnp.save("y_train.npy", y_train)\nnp.save("y_test.npy", y_test)\n'

In [9]:
y_train[1000:2000].shape

(1000,)

Обучение:

In [8]:
clf = Bayes_Wrapper(alpha=1.0, num_answers=1, answers="one")
clf.fit(X_train, y_train)

(100, 12071) (100,)
<class 'scipy.sparse.csr.csr_matrix'> <class 'numpy.ndarray'>


MemoryError: 